In [1]:
# import os
# from bs4 import BeautifulSoup, NavigableString, Tag 

from nltk.util import ngrams
from nltk import word_tokenize

# import json
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as plt
import seaborn as sns

import collections
import csv
# from sets import Set

from datetime import datetime
from pytz import timezone

print("imported")

imported


In [2]:
# IMPORTS SPECIFIC TO TOPIC MODELING

# import langid
import nltk
import re
import time
from collections import defaultdict
from configparser import ConfigParser
from gensim import corpora, models, similarities
from nltk.tokenize import RegexpTokenizer
from string import digits
from nltk.stem import WordNetLemmatizer
print("imported")

imported


import editorial text

In [3]:
import import_func as imp
import tags_work as tgs
import desc_vis as vis
import classifier_func as cls
import classifier_help as clh

### Loading Documents and Splitting Into Lines

In [4]:
# full_advice = "../data/genre_advice_full_021520.jl"
full_filename = "../data/by_article_fulltext_020920.jl"

# get full text dataset as a df
# advice_df = imp.init_df(full_advice, "full", genre="advice")
# advice_df = advice_df[advice_df["year"]<2020]
full_df = imp.init_df(full_filename, "full")
full_df = full_df[full_df["year"]<2020]

full_df.head(3)

,id,headline,tags,authors,date,time,text,bio,date_seq,month_seq,year,author,n_posts_author,column1,column2,advice,one_time
0,5839,how to write a winning résumé,"[tooling up, advice, graduate, academic, indus...",peter fiske,1996-10-18,8:00 am,"\n by welcome to ""tooling up,"" a monthly col...",[peter fiske is a ph.d. scientist and co-found...,18,10,1996,peter fiske,59,no,yes,yes,no
1,5835,the commandments of cover letter creation,"[tooling up, advice, americas]",peter fiske,1996-12-20,0:00 am,"\n by \n 've always hated the term ""cover le...",[peter fiske is a ph.d. scientist and co-found...,81,12,1996,peter fiske,59,no,yes,yes,no
2,5840,dressing scientists for success: male case study,"[tooling up, advice, graduate, postdoc, academ...",peter fiske,1997-09-26,8:00 am,"\n by \n e all know that, with respect to fa...",[peter fiske is a ph.d. scientist and co-found...,361,21,1997,peter fiske,59,no,yes,yes,no


In [256]:
#LOAD DOCUMENT BY LINE INTO AN ARRAY

# files = [filename.split('.')[0] for filename in os.listdir('digitalSTS/papers/txt/') if not filename.startswith('.')]
# print files

line_num = [i for i in range(len(full_df))]
filenames = ["advice","news","wl"]

for num in line_num:

    outpath = 'digitalSTS/papers/lines/' + filename + '.csv'
    w_line = full_df.iloc[line]
    
    with open(outpath, 'w') as outfile:
        
#         for line in infile.readlines():
#             print line
        paragraphs = infile.readlines()
    
        for paragraph in paragraphs:

            paragraph = paragraph.decode('unicode_escape').encode('ascii','ignore')
            lines = [line.strip().lower() for line in ''.join(paragraph).split('.')]
            #             print paragraph
            
            writer = csv.writer(outfile)

            for line in lines:
                words = line.split(' ')
                if len(words) > 1:
                    writer.writerow([line.encode('utf-8')])

In [277]:
files = [filename.split('.')[0] for filename in os.listdir('digitalSTS/papers/csvs/') if not filename.startswith('.')]
print files

['02_Parmiggiani_Monteiro', '03_Allhutter', '04_Camus_Vinck', '05_Lynch', '06_Vertesi', '07_Kerisadou', '08_Couture', '09_Dunbar_Hester', '10_Hawthorne', '11_Chan', '12_Poster', '13_Ilten_McInerney', '14_Nemer', '15_Erikson', '16_Toth', '17_Cohn', '18_Seaver', '19_Stark', '20_Ribes', '21_Cardoso_Llach', '22_Venturini', '23_Salamanca', '24_Munk', '25_Calvillo', '26_Winthereik', '27_Loukissas']


### Creating Dictionaries and Corpuses

In [279]:
files = [filename.split('.')[0] for filename in os.listdir('digitalSTS/papers/csvs/') if not filename.startswith('.')]
# print files

for filename in files:

    inpath = 'digitalSTS/papers/csvs/' + filename + '.csv'
    
    with open(inpath, 'r') as infile:
        
        reader = csv.reader(infile)
        documents = [line[0] for line in reader]
#         print lines

        # TOKENIZING
        tokenizer = RegexpTokenizer(r'\w+')
        documents = [tokenizer.tokenize(doc.lower()) for doc in documents ]
        
        # GATHERING CANDIDATES FOR CUSTOM LIST OF STOPWORDS FROM NGRAMS
        # ['i','m','a','a','i','t','6',...]
        unigrams = [ w for doc in documents for w in doc if len(w)==1]
        # ['do','or','or','of','be',...]
        bigrams  = [ w for doc in documents for w in doc if len(w)==2]

        misc = ['get', 'know', 'would']
        
        # CREATES THE SET OF UNIQUE "CUSTOM STOP LIST WITH NLTK + THE ABOVE
        stoplist  = set(nltk.corpus.stopwords.words("english") + unigrams + bigrams + misc)

        # REMOVES STOP WORDS
        documents = [[token for token in doc if token not in stoplist]
                        for doc in documents]

        # REMOVES WORDS THAT ARE NUMBERS ONLY
        documents = [ [token for token in doc if len(token.strip(digits)) == len(token)]
                        for doc in documents ]
        
        # REMOVE WORDS THAT OCCUR ONLY ONCE
        token_frequency = defaultdict(int)

        # count all tokens
        for doc in documents:
            for token in doc:
                token_frequency[token] += 1

        # keep words that occur more than once
        documents = [ [token for token in doc if token_frequency[token] > 1]
                        for doc in documents  ]

        # Sort words in documents
        for doc in documents:
            doc.sort()
            
        # I HATE UNICODE!!
        documents = [ [token.decode('ascii', 'ignore') for token in doc] for doc in documents  ]
        
        # Build a dictionary where for each document each word has its own id
        dictionary = corpora.Dictionary(documents)
        dictionary.compactify()
        # and save the dictionary for future use
        print dictionary
        dictionary.save( 'digitalSTS/papers/dicts/' + filename + '.dict')
        
        # BUILD AND SAVE CORPUS
        corpus = [dictionary.doc2bow(doc) for doc in documents]
        corpora.MmCorpus.serialize('digitalSTS/papers/corpuses/' + filename + '.mm', corpus)

Dictionary(873 unique tokens: [u'represent', u'concept', u'managed', u'forbidden', u'lack']...)
Dictionary(821 unique tokens: [u'represent', u'concept', u'semantic', u'grassroots', u'scratch']...)
Dictionary(948 unique tokens: [u'concept', u'semantic', u'consider', u'magnetic', u'dynamic']...)
Dictionary(784 unique tokens: [u'limited', u'code', u'forbidden', u'lack', u'focus']...)
Dictionary(955 unique tokens: [u'represent', u'todays', u'consider', u'focus', u'yellow']...)
Dictionary(782 unique tokens: [u'concept', u'managed', u'focus', u'four', u'woods']...)
Dictionary(644 unique tokens: [u'limited', u'code', u'consider', u'chain', u'focus']...)
Dictionary(698 unique tokens: [u'limited', u'code', u'illustrates', u'hooks', u'oss']...)
Dictionary(772 unique tokens: [u'represent', u'concept', u'consider', u'limited', u'dissolution']...)
Dictionary(794 unique tokens: [u'limited', u'code', u'managed', u'global', u'dynamic']...)
Dictionary(1245 unique tokens: [u'represent', u'concept', u'pa

In [266]:
# CODE HELP FROM:
# https://github.com/alexperrier/datatalks/blob/master/twitter/twitter_preprocessing.py

# SPLITS EACH POST INTO WORDS WHILE REMOVING PUNCTUATION (\w+) AND LOWER CASES THEM A WORD SUCH AS
# I'M WILL BECOME I, M
tokenizer = RegexpTokenizer(r'\w+')
documents = [tokenizer.tokenize(doc.lower()) for doc in documents ]
## [['what', 'do', 'they', 'look', 'for', 'for', 'approval', 'or', 'denial'],
##  ['hi','i','m','wondering','if','someone','could','answer',...]...
documents[:2]

[['digitized', 'coral', 'reefs'],
 ['elena', 'parmiggiani', 'and', 'eric', 'monteiro']]

In [267]:
# I'M NOT SURE THESE ARE TECHNICALLY UNIGRAMS AND BIGRAMS SO MUCH AS THEY ARE SINGLE LETTER WORDS
# AND TWO LETTER WORDS BUT WHO AM I TO JUDEGE (SHULDER SHURG)

# ['i','m','a','a','i','t','6',...]
unigrams = [ w for doc in documents for w in doc if len(w)==1]
# ['do','or','or','of','be',...]
bigrams  = [ w for doc in documents for w in doc if len(w)==2]

misc = ['get', 'know', 'would']

print unigrams[:5]
print bigrams[:5]

['a', 'a', 'a', 'a', 'a']
['mr', 'in', 'is', 'in', 'on']


In [268]:
# CREATES THE SET OF UNIQUE "CUSTOM STOP LIST WITH NLTK + THE ABOVE
stoplist  = set(nltk.corpus.stopwords.words("english") + unigrams + bigrams + misc)

# REMOVES STOP WORDS
documents = [[token for token in doc if token not in stoplist]
                for doc in documents]

# REMOVES WORDS THAT ARE NUMBERS ONLY
documents = [ [token for token in doc if len(token.strip(digits)) == len(token)]
                for doc in documents ]

# # [['look', 'approval', 'denial', 'parole', 'approval'...],
# # ['wondering', 'someone', 'could', 'answer', 'question'...]...]
print documents[0][:5]
print documents[1][:5]

['digitized', 'coral', 'reefs']
['elena', 'parmiggiani', 'eric', 'monteiro']


In [269]:
# REMOVE WORDS THAT OCCUR ONLY ONCE
token_frequency = defaultdict(int)

# count all tokens
for doc in documents:
    for token in doc:
        token_frequency[token] += 1

# keep words that occur more than once
documents = [ [token for token in doc if token_frequency[token] > 1]
                for doc in documents  ]

# Sort words in documents
for doc in documents:
    doc.sort()
    
## [['approval', 'approval', 'approval', 'approval', 'approval'...],
##  ['answer', 'benefit', 'benefit', 'benefit', 'confused'...]...]

print documents[0][:5]
print documents[1][:5]

['coral', 'digitized', 'reefs']
['monteiro', 'parmiggiani']


In [270]:
# I HATE UNICODE!!
documents = [ [token.decode('ascii', 'ignore') for token in doc] for doc in documents  ]
print documents[0][:5]
print documents[1][:5]

[u'coral', u'digitized', u'reefs']
[u'monteiro', u'parmiggiani']


In [271]:
# Build a dictionary where for each document each word has its own id
dictionary = corpora.Dictionary(documents)
dictionary.compactify()
# and save the dictionary for future use
dictionary.save('parmiggiani.dict')

In [272]:
# We now have a dictionary with 11127 unique tokens
# Dictionary(11127 unique tokens: [u'foul', u'narcotic', u'four', u'woods', u'hanging']...)
print dictionary

Dictionary(873 unique tokens: [u'represent', u'concept', u'managed', u'forbidden', u'lack']...)


In [273]:
# Build the corpus: vectors with occurence of each word for each document
# convert tokenized documents to vectors
corpus = [dictionary.doc2bow(doc) for doc in documents]

In [274]:
# and save in Market Matrix format
corpora.MmCorpus.serialize('parmiggiani.mm', corpus)
# this corpus can be loaded with corpus = corpora.MmCorpus('prison_talk_nyc.mm')

## MISC

### Lemmatization

In [58]:
files = [filename.split('.')[0] for filename in os.listdir('../papers/strings/') if not filename.startswith('.')]
# files

In [84]:
# filename = '02_Parmiggiani_Monteiro'
# testpath = '../papers/tests/' + '02_Parmiggiani_Monteiro' + '.csv'

totalfq = collections.Counter()

for filename in files:

    inpath = '../papers/strings/' + filename + '.csv'
    outpath = '../papers/lemmas/' + filename + '.csv'
    
    with open(inpath, 'r') as infile, open(outpath, 'w') as outfile:

        essay = infile.readline()

        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(essay.lower())

        # ['i','m','a','a','i','t','6',...]
        unichars = [word for word in tokens if len(word)==1]
        # ['do','or','or','of','be',...]
        bichars  =  [word for word in tokens if len(word)==2]

        misc = ['get', 'know', 'would', 'and', 'yet', 'due', 'one', 'digital', 'technology', 'way', \
                'also', 'use', 'within', 'like', 'sts', 'may', 'however', 'even', u'technology', 'used']

        stoplist  = set(nltk.corpus.stopwords.words("english") + unichars + bichars + misc)

        # REMOVES WORDS THAT ARE NUMBERS ONLY
        post_stop = [token for token in tokens if len(token.strip(digits)) == len(token)]

        # REMOVES STOP WORDS
        post_stop = [token for token in tokens if token not in stoplist]

        post_lemma = [lemma.lemmatize(token) for token in post_stop]

        tokenfq = collections.Counter()
        tokenfq.update(post_lemma)
        
        post_highfq = [token for token in post_lemma if tokenfq[token] > 1]

        totalfq.update(post_highfq)
        
        writer = csv.writer(outfile)
        writer.writerow([' '.join(post_highfq).encode('utf-8')])

        print filename, '----> TOKENS IN:', len(tokens), 'POST STOP:', len(post_stop), 'POST HIGHFQ:', len(post_highfq), \
                        '----> PCT:', round(float(len(post_highfq))/float(len(tokens)), 2) * 100

02_Parmiggiani_Monteiro ----> TOKENS IN: 7519 POST STOP: 3991 POST HIGHFQ: 3136 ----> PCT: 42.0
03_Allhutter ----> TOKENS IN: 4129 POST STOP: 2403 POST HIGHFQ: 1719 ----> PCT: 42.0
04_Camus_Vinck ----> TOKENS IN: 10563 POST STOP: 5268 POST HIGHFQ: 4356 ----> PCT: 41.0
05_Lynch ----> TOKENS IN: 7075 POST STOP: 3435 POST HIGHFQ: 2575 ----> PCT: 36.0
06_Vertesi ----> TOKENS IN: 9048 POST STOP: 4755 POST HIGHFQ: 3811 ----> PCT: 42.0
07_Kerisadou ----> TOKENS IN: 6299 POST STOP: 3201 POST HIGHFQ: 2387 ----> PCT: 38.0
08_Couture ----> TOKENS IN: 7745 POST STOP: 3897 POST HIGHFQ: 3234 ----> PCT: 42.0
09_Dunbar_Hester ----> TOKENS IN: 4436 POST STOP: 2392 POST HIGHFQ: 1541 ----> PCT: 35.0
10_Hawthorne ----> TOKENS IN: 5563 POST STOP: 3151 POST HIGHFQ: 2241 ----> PCT: 40.0
11_Chan ----> TOKENS IN: 5448 POST STOP: 3207 POST HIGHFQ: 2308 ----> PCT: 42.0
12_Poster ----> TOKENS IN: 13320 POST STOP: 7193 POST HIGHFQ: 5922 ----> PCT: 44.0
13_Ilten_McInerney ----> TOKENS IN: 4999 POST STOP: 2837 POST 